In [1]:
import sys
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import lightgbm as lgb
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error

import logging
logging.basicConfig(level=logging.ERROR)

# 変数

In [2]:
preprocess_num = "P1"

# csvをimportする

In [3]:
train = pd.read_csv(f"../preprocess_results/{preprocess_num}_train.csv", sep=',')
train = train.drop(columns = ["id"])
train.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,1,1,1,7.0,1,0,2,0,11.611723,112.15875
1,1,0,2,10.0,1,1,1,3,27.078537,68.88056
2,4,1,2,2.0,1,0,1,5,16.643760,39.17320
3,2,2,2,8.0,1,0,1,3,12.937220,80.60793
4,0,0,1,1.0,1,1,1,3,17.749338,86.02312


In [4]:
test = pd.read_csv(f"../preprocess_results/{preprocess_num}_test.csv", sep=',')
test = test.drop(columns = ["id","Price"])
test.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
0,3,1,2,2.0,0,0,2,3,20.671147
1,2,0,1,7.0,0,1,0,3,13.564105
2,0,0,0,9.0,0,1,1,1,11.809799
3,0,2,0,1.0,1,0,1,3,18.477036
4,5,2,0,2.0,1,1,2,0,9.907953


In [5]:
sample_submission = pd.read_csv(f"../data/sample_submission.csv", sep=',')
sample_submission.head()

,id,Price
0,300000,81.411
1,300001,81.411
2,300002,81.411
3,300003,81.411
4,300004,81.411


# 学習データとテストデータに分割

In [6]:
X = train.drop(columns=['Price'])
y = train["Price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# パラメーターチューリング

In [7]:
def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "boosting_type": "gbdt",
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),
        "num_leaves": trial.suggest_int("num_leaves", 20, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 10.0),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 10.0),
        "n_estimators": 1000,
        "early_stopping_rounds": 10,
        "verbose" : 0
    }

    # 交差検証
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmse_list = []

    for train_idx, valid_idx in kf.split(X_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]

        # モデルの学習
        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            eval_metric="rmse",
        )
        
        # 予測
        y_pred = model.predict(X_val)

        # RMSEの計算
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        rmse_list.append(rmse)

    return np.mean(rmse_list)

In [8]:
%%time

# Optunaで最適化
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# 最適なパラメータ
best_params = study.best_params
print("Best Parameters:", best_params)

[I 2025-02-18 23:41:57,695] A new study created in memory with name: no-name-11b1d249-2455-4ab0-840d-387d15429793
C:\Users\ns_m0\AppData\Local\Temp\ipykernel_2840\734631466.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),
C:\Users\ns_m0\AppData\Local\Temp\ipykernel_2840\734631466.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "subsample": trial.suggest_uniform("subsample", 0.5, 1.0),
C:\Users\ns_m0\AppData\Local\Temp\ipykernel_2840\734631466.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use

[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


C:\Users\ns_m0\Downloads\anaconda\envs\kaggle\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


C:\Users\ns_m0\Downloads\anaconda\envs\kaggle\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


...

[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


C:\Users\ns_m0\Downloads\anaconda\envs\kaggle\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


C:\Users\ns_m0\Downloads\anaconda\envs\kaggle\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-02-18 23:49:19,031] Trial 49 finished with value: 39.02694515767832 and parameters: {'learning_rate': 0.05141814791486403, 'num_leaves': 21, 'max_depth': 10, 'min_child_samples': 8, 'subsample': 0.6074790554888865, 'colsample_bytree': 0.590257969865659, 'reg_alpha': 7.458231535092952e-05, 'reg_lambda': 3.626736479791056e-05}. Best is trial 21 with value: 39.02512657078061.


Best Parameters: {'learning_rate': 0.047821797536697096, 'num_leaves': 48, 'max_depth': 8, 'min_child_samples': 6, 'subsample': 0.5882344539087228, 'colsample_bytree': 0.5974974154361233, 'reg_alpha': 9.10987878298426, 'reg_lambda': 3.841378850507732e-06}
CPU times: total: 21min 53s
Wall time: 7min 21s


# GBDT

In [9]:
# 最適なパラメータでモデルを学習
best_model = lgb.LGBMRegressor(**best_params)
print(f"best_model: {best_model}")
kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X_train))

for train_idx, valid_idx in kf.split(X_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]  # ilocを使用
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]  # 同様にy_trainもilocでアクセス

    best_model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        eval_metric="rmse",
    )

    oof_preds[valid_idx] = best_model.predict(X_val)

# OOFスコア
rmse_oof = mean_squared_error(y_train, oof_preds, squared=False)
print(f"OOF RMSE: {rmse_oof:.4f}")

best_model: LGBMRegressor(colsample_bytree=0.5974974154361233,
              learning_rate=0.047821797536697096, max_depth=8,
              min_child_samples=6, num_leaves=48, reg_alpha=9.10987878298426,
              reg_lambda=3.841378850507732e-06, subsample=0.5882344539087228)
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underline

C:\Users\ns_m0\Downloads\anaconda\envs\kaggle\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [10]:
# テストデータで評価
y_test_pred = best_model.predict(X_test)
rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)
print(f"Test RMSE: {rmse_test:.4f}")

Test RMSE: 38.9053


C:\Users\ns_m0\Downloads\anaconda\envs\kaggle\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [11]:
# 本番データで予測出し
y_prod_pred = best_model.predict(test)

# csvをmodel_resultsに作成

In [12]:
sample_submission["Price"] = y_prod_pred
display(sample_submission.head())

path = f"../model_results/ML1.csv"
sample_submission.to_csv(path, index=False)

,id,Price
0,300000,81.989644
1,300001,82.638587
2,300002,82.283590
3,300003,80.733604
4,300004,78.850761
